# Watson OpenScale Mortgage Default Lab - Model Feed

Part two of a hands-on lab for IBM Watson OpenScale, this notebook should be run in a Watson Studio project with Python 3.6 or greater. This notebook will use the Watson Machine Learning client to send scoring requests to the model. You can specify the number of requests sent in the cells below.

Replace the credentials below with those of your Watson Machine Learning instance with the mortgage default model deployed.

In [1]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud",
    "username": "admin",
    "password": "********"
}

In [2]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In the cell below, ensure that the model and deployment names are the same used for creating the model, and set the minimum and maximum number of records you wish to send to the model for scoring.

In [3]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

MIN_PAYLOADS = 101
MAX_PAYLOADS = 125

Using the credentials from above, create a Watson Machine Learning API client.

In [5]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.70'

Get the deployment and model IDs from the WML client.

In [7]:
space_name = "Mortage Default Production"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
        print('Setting deployment space...')
        wml_client.set.default_space(space_id)
        

Setting deployment space...


In [8]:
wml_client.repository.list_models()

------------------------------------  -----------------------------  ------------------------  -----------------
ID                                    NAME                           CREATED                   TYPE
45266e71-0bca-42bb-9dbf-3094d0d1cb84  Mortgage Default - Production  2021-04-14T22:15:04.002Z  scikit-learn_0.23
------------------------------------  -----------------------------  ------------------------  -----------------


In [14]:
deployment_id = None
for deployment in wml_client.deployments.get_details()['resources']:
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        deployment_id = deployment['metadata']['id']
print(deployment_id)

Mortgage Default - Production
WOS-INTERNAL-1f6dd3fa-16b2-449f-8ab3-97246235443e
9d8d6e03-7092-4aa6-8d93-1566fdea1c73


In [ ]:
for deployment in wml_client.deployments.get_details()['resources']:
    deployment_id=deployment['metadata']['id']
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid)

In [15]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

--2021-04-14 22:40:01--  https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32112 (31K) [text/plain]
Saving to: ‘mortgage_feed.json’

mortgage_feed.json  100%[===================>]  31.36K  --.-KB/s    in 0.001s  

2021-04-14 22:40:02 (50.6 MB/s) - ‘mortgage_feed.json’ saved [32112/32112]



In [16]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)

In [21]:
scoring_payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'][1:],
                           "values": [],
                           "meta": {
                               "fields": ["ID"],
                               "values": []
                           }
                       }
                    ]
                  }

In [22]:
import random
import string

letters = string.digits

for _ in range(0, 101):
    value_to_score = random.choice(data['values'])
    scoring_payload['input_data'][0]['values'].append(value_to_score[1:])
    scoring_payload['input_data'][0]['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])
print(len(scoring_payload['input_data'][0]['values']))

101


In [23]:
predictions = wml_client.deployments.score(deployment_id, scoring_payload)

In [24]:
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['DENIED', [0.10666666666666667, 0.8933333333333333]],
    ['APPROVED', [0.85, 0.15]],
    ['APPROVED', [0.96, 0.04]],
    ['DENIED', [0.15, 0.85]],
    ['APPROVED', [0.8975, 0.1025]],
    ['APPROVED', [0.82, 0.18]],
    ['APPROVED', [0.64, 0.36]],
    ['APPROVED', [0.9, 0.1]],
    ['DENIED', [0.18, 0.82]],
    ['APPROVED', [0.9566666666666667, 0.04333333333333334]],
    ['DENIED', [0.19, 0.81]],
    ['APPROVED', [0.54, 0.46]],
    ['APPROVED', [0.74, 0.26]],
    ['DENIED', [0.1, 0.9]],
    ['DENIED', [0.17, 0.83]],
    ['APPROVED', [0.856, 0.14400000000000002]],
    ['APPROVED', [0.81, 0.19]],
    ['DENIED', [0.23, 0.77]],
    ['APPROVED', [0.74, 0.26]],
    ['APPROVED', [0.92, 0.08]],
    ['APPROVED', [0.8775, 0.1225]],
    ['DENIED', [0.04, 0.96]],
    ['APPROVED', [0.9, 0.1]],
    ['DENIED', [0.4108333333333333, 0.5891666666666667]],
    ['APPROVED', [0.94, 0.06]],
    ['DENIED', [0.23, 0.77]],
    ['APPROVED'